# 05-3 트리의 앙상블

**CSV 파일에 가지런이 정리되어 있는 형태의 데이터를 정형 데이터(structured data)라고 부름**
- 이런 데이터는 CSV나 데이터베이스(Database), 혹은 엑셀(Excel)에 저장하기 쉬움

**정형 데이터와 반대되는 데이터를 비정형데이터(unstructured data)라고 부름**
- DB나 엑셀로 표현하기 어려운 사진, 음악 등임

**정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습(ensemble learning)임**
- 이 알고리즘은 대부분 결정 트리를 기반으로 만들어짐

비정형 데이터는 신경망 알고리즘을 사용해야 함

**랜덤 포레스트 (Random Forest)는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용**됨
- 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듬
- 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듬

먼저 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 독특한 방식을 사용
- **우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듬 (한 샘플이 중복되어 추출될 수 있음)**
- 1000개의 샘플이 들어 있는 가방에서 100개의 샘플을 뽑는다면, 1개를 뽑고 다시 가방에 넣고 이런식으로 100개를 뽑으면 중복된 샘플을 뽑을 수 있음
- **이렇게 만들어진 샘플을 부트스트랩 샘플 (bootstrap sample)이라고 함**
 - 기본적으로 부트스트랩 샘플은 훈련 세트와 크기가 같게 만듬
 - 1000개의 샘플이 있는 가방에서 중복하여 1000개의 샘플을 뽑음

**또한 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음**
- 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
- 즉 4개의 특성이 있다면, 노드마다 2개를 랜덤하게 선택하여 사용
- 다만, 회귀모델인 RandomForestRegressor는 전체 특성을 사용

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련함
- 분류인 경우 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음
- 회귀일 때는 단순히 각 트리의 예측을 평균함

**랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음**
- 종종 기본 매개변수 설정만으로도 아주 좋은 결과를 냄

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

cross_validate() 함수를 사용해 교차검증 수행
- RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어를 사용하는 것이 좋음
- cross_validate() 함수의 n_jobs 매개변수도 -1로 지정하여 최대한 병렬로 교차검증 수행
- return_train_score 매개변수를 True로 지정하여 훈련 세트에 대한 점수도 같이 반

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


출력된 결과는 훈련 세트에 다소 과대적합됨, 이 예제는 매우 간단하고 특성이 많지 않아 그리드 서치를 사용하더라도 하이퍼파라미터 튜닝의 결과가 크게 나아지지 않음

In [3]:
# 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 맥개변수를 모두 제공
# 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


[알코올 도수, 당도, pH]

결정 트리에서의 특성 중요도와 비교 했을 때, 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승함
- 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문
- 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특서잉 훈련에 기여할 기회를 얻음
- 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움을 줌

**RandomForestClassfier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있음**
- 랜덤 포레스트는 부트스트랩 샘플을 만들어 결정 트리를 훈련함
- **이때 부트스트랩 샘플에 포함되지 않고 남는 샘플을 OOB(out of bag)샘플이라고 함**
- 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음 (검증 세트의 역할)
- 이 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정해야 함
 - 이렇게 하면 랜덤 포레스트는 각 결정 트리의 OOB 점수를 평균하여 출력
 - OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 훈련 세트에 더 많은 샘플 사용 가능

In [10]:
# OOB 점수 출력

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


**엑스트라 트리(Extra Trees)는 랜덤 포레스트와 매우 비슷하게 동작함**
- 기본적으로 100개의 결정 트리 훈련
- 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수 지원
- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용
- **랜덤 포레스트와 차이 점은 부트스트랩 샘플을 사용하지 않는 점**
 - 즉, 결정 트리를 만들 때 전체 훈련 세트를 사용
 - 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함
 - 엑스트라 트리가 사용하는 결정 트리가 바로 splitter='random'인 결정 트리
 - 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만, 많은 트리를 앙상블 하기때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음
 - 보통 엑스트라 트리가 무작위성이 좀 더 크기때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야함
 - 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 장
 - 엑스트라 트리의 회귀 버전은 ExtraTreesRegressor 클래스

In [12]:
# 사이킷런에서 제공하는 엑스트라 트리는 ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [13]:
# 알코올 도수, 당도, pH 특성 중요도

et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


**그레이디언트 부스팅(gradient boosting)은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법**
- 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능 기대 가능
- 경사하강법을 사용하여 트리를 앙상블에 추가함
- 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용
- 그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동
- 학습률 매개변수로 속도 조

In [14]:
# 그레이디언트 부스팅

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함

학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능 향상 가능

In [15]:
# 결정 트리의 개수를 500개(5배), 학습률의 기본값은 (0.1)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그레이디언트 부스팅이 랜덤 포레스트보다 일부 특성(당도)에 더 집중

**subsample은 트리 훈련에 사용할 훈련 세트의 비율을 정하는 매개변수**
- 기본 값은 1.0으로 전체 훈련 세트를 사용
- 하지만 1보다 작으면 훈련 세트의 일부를 사용함 (확률적 경사하강법, 미니배치 경사하강법과 비슷)

**일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음**
- 하지만 순서대로 트리를 추가하기 때문에 훈련속도가 느림
- 즉 GradientBoostingClassifier에는 n_jobs 매개변수가 없음
- 그레이디언트 부스팅의 회귀 버전은 GradientBoostingRegressor
- 그레이디언트 부스팅의 속도와 성능을 더욱 개선한 것이 히스토그램 기반 그레이디언트 부스팅

**히스토그램 기반 그레이디언트 부스팅 (Gistogram-based Gradient Boosting)은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높음**
- 히스토그램 기반 그레이디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 나눔
 - 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
 - 히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용
 - 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음
- 사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier임
 - HistGradientBoostingClassifier에는 트리의 개수를 지정하는데 n_estimators 대신에 부스팅 반복 횟수를 지정하는 max_iter를 사용

In [17]:
# from sklearn.experimental import enable_hist_gradient_boosting (사이킷런 1.0 버전 아래에서는 주석 해제 후 실행 필요)

from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


**히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 <font color ="red">permutation_importance() 함수</font>를 사용**
- 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산함
- 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있음

In [18]:
# n_repeats 매개변수는 랜덤하게 섞을 횟수 지정 (기본값은 5)

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


**permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)를 담고 있음**
- 평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있음

In [19]:
# 테스트 세트의 특성 중요도 계산

result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [20]:
# 테스트 세트에서의 성능 최종적 확인

hgb.score(test_input, test_target)

0.8723076923076923

앙상블 모델은 단일 결정 트리보다 좋은 결과를 얻을 수 있음 (1%차이)

사이킷런 말고도 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여러개 존재
- **가장 대표적인 라이브러리는 XGBoost**
 - 코랩에서 사용할 수 있을 뿐만 아니라 사이킷런의 cross_validate() 함수와 함께 사용 가능
 - 다양한 부스팅 알고리즘 지원
   - tree_method 매개변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅 사용 가능

In [21]:
# XGBoost

from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


**널리 사용하는 또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 마이크로소프트에서 만든 LifghtGBM^2**
- LightGBM은 빠르고 최신 기술을 많이 적용하고 있어 인기가 점점 높아지는 중


In [22]:
# LightGBM

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
